In [6]:
from random import *
from math import *
from scipy.optimize import minimize
from scipy.stats import gamma 
from cmath import pi 

import numpy as np 



SQRTPI: float = 1.77245385090551602729816748334
SQRT2PI: float = 2.50662827463100029

EULER_MASCHERONI: float = 0.57721566490153286060



def gaussian_pdf(x) -> float:
  return exp(-x * x / 2.0) / SQRT2PI


def gaussian_cdf(x)->float:
  if x >= 0.0:
    t=1.0/(1.0 + 0.33267 * x)
    return 1.0 - gaussian_pdf(x) * (0.4361836*t-0.1201676*t*t + 0.9372980*t*t*t)
  else:
    t=1.0 / (1.0-0.33267*x);
    return gaussian_pdf(x) * (0.4361836*t-0.1201676*t*t + 0.9372980*t*t*t)


def gaussian_sdf(x)->float:
  if x>=0.0:
    t=1.0/(1.0+0.33267*x)
    return gaussian_pdf(x)(0.4361836*t-0.1201676*t*t+0.9372980*t*t*t)
  else:
    t=1.0/(1.0-0.33267*x);
    return 1.0-gaussian_pdf(x) * (0.4361836*t-0.1201676*t*t+0.9372980*t*t*t)


def gaussian_invcdf(u)->float:
  if u<0.5:
    t=sqrt(log(1.0/(u*u)))
    return -t+(2.515517+0.802853*t+0.010328*t*t)/(1.0+1.432788*t+0.189269*t*t+0.001308*t*t*t)
  elif u==0.5:
    return 0.0
  else:
    t=sqrt(log(1.0/((1.0-u)*(1.0-u))))
    return t - (2.515517 + 0.802853*t + 0.010328*t*t) / (1.0+1.432788*t+0.189269*t*t + 0.001308*t*t*t)









def uni_uniform_D(r):
  return uniform(1-r,1)







def uni_uniform_E(r):
  return uniform(0,1-r)







def bi_uniform_D(r):
  u0=uniform(0,1)*(2-r)
  u1=uniform(0,1)
  u2=uniform(0,1)
  v1=0
  v2=0
  if u0<(1-r):
    v1=1-(r * u1)
    v2=(1-r) * u2 
  elif u0 < 2*(1-r):
    v1=(1-r)*u1
    v2=1-(r*u2) 
  else:
    v1=1-(r*u1)
    v2=1-(r*u2) 
  return v1, v2 






def bi_uniform_E(r):
  u1=uniform(0,1-r)
  u2=uniform(0,1-r)
  return u1, u2 








class Distribution:
    def __init__(self, name, mean, stdev):
        self.name: str=name 
        self.mean: float=mean 
        self.stdev: float=stdev 
        self.min: float=-np.inf 
        self.max: float=np.inf 

    def print_name(self):
        print(self.name, "(", self.mean, ",", self.stdev, ")")

    def get_name(self):
        namestr=self.name+ "({0}, {1})"
        return namestr.format(self.mean, self.stdev)

    def getMean(self)->float:
        return self.mean 
    
    def getStdev(self)->float: 
        return self.stdev 
    
    def getMin(self)->float: 
        return self.min 
    
    def getMax(self)->float:
        return self.max 

    def getPDF(self, x)->float:
        pass 

    def getCDF(self, x)->float: 
        pass 

    def getSDF(self, x) -> float:
        pass

    def getHazardRate(self, x)->float:
        f=self.getPDF(x)
        s=self.getSDF(x)
        if s>0:
            return f/s
        else:
            return 0

    def lowerPercentile(self, p)-> float:
        pass 

    def getLowerPercentile(self, p)-> float: 
        if p<=0:
            return self.min 
        elif p>=1:
            return self.max 
        else:
            return self.lowerPercentile(p)

    def getUpperPercentile(self, p)->float:
        if p<=0:
            return self.max 
        elif p>=1:
            return self.min 
        else:
            return self.lowerPercentile(1-p)

    def getStochasticValue(self)->float: 
        u=uniform(0,1)
        return self.getLowerPercentile(u)













class Pareto(Distribution):
    def __init__(self, mean, stdev):
        super().__init__("PARETO", mean, stdev)
        self.tau: float =sqrt((mean * mean) / (stdev*stdev) + 1)+1
        self.Xm: float =mean * (self.tau - 1) / self.tau 
        self.min=self.Xm 
        print(self.name, "(mean=", self.mean, ", stdev=", self.stdev, ", tau =", self.tau, ",Xm=", self.Xm, ")") 

    def getPDF(self, x)-> float: 
        if x>self.min:
            return self.tau*(self.min**self.tau)/(x**(self.tau+1))
        else:
            return 0

    def getCDF(self, x)->float:
        if x>self.min: 
            return 1-(self.min / x)**self.tau 
        else:
            return 0

    def getSDF(self, x)-> float:
        if x>self.min:
            return(self.min/x)**self.tau 
        else:
            return 1

    def lowerPercentile(self, p)-> float:
        return self.Xm*((1-p)**(-1/self.tau))







class Lognormal(Distribution):
    def __init__(self, mean, stdev):
        super().__init__("LOGNORMAL", mean, stdev)
        self.h = 1.0 + (stdev * stdev) / (mean * mean)
        self.sigma=sqrt(log(self.h))
        self.mu=log(self.mean)-0.5*log(self.h)
        self.min=0
        print(self.name, "(mean=", self.mean, ", stdev=", self.stdev, ", logmean=", self.mu, ", logstdev=", self.sigma, ")") 

    def getPDF(self, x)-> float:
        if x>self.min: 
            return gaussian_pdf((log(x)-self.mu)/self.sigma)/(self.sigma*x)
        else:
            return 0

    def getCDF(self, x)-> float:
        if x>self.min:
            return gaussian_cdf((log(x)-self.mu)/self.sigma)
        else:
            return 0

    def getSDF(self, x)-> float: 
        if x>self.min: 
            return gaussian_sdf((log(x)-self.mu)/self.sigma)
        else:
            return 1
    
    def lowerPercentile(self, p)-> float:
        return self.mean*exp(self.sigma*gaussian_invcdf(p)/sqrt(self.h))








class Truncnormal(Distribution):
    def __init__(self, mean, stdev):
        super().__init__("TNORMAL", mean, stdev)
        alpha=find_alpha(mean, stdev)
        self.sigma=mean*(1-gaussian_cdf(alpha))/(gaussian_pdf(alpha)-alpha*(1-gaussian_cdf(alpha)))
        self.mu= -alpha*self.sigma 
        self.omega=gaussian_cdf(-self.mu/self.sigma)
        self.min=0
        print(self.name,"(mean=", self.mean, ", stdev=", self.stdev, ", sigma=", self.sigma, ", mu=", self.mu, ")") 

    def getPDF(self, x)->float:
        if x>self.min:
          return (gaussian_pdf((x-self.mu)/self.sigma)/self.sigma)/(1-self.omega)
        else:
          return 0

    def getCDF(self, x)-> float:
        return 1-self.getSDF(x)
    
    def getSDF(self, x)->float:
        if x>self.min:
            return gaussian_sdf((x-self.mu)/self.sigma)/(1-self.omega)
        else:
            return 1

    def lowerPercentile(self, p)->float:
        return self.mu+self.sigma*gaussian_invcdf(self.omega+ p*(1-self.omega))








class Gamma(Distribution):
    def __init__(self, mean, stdev):
        super().__init__('GAMMA', mean, stdev)
        self.tau=(mean*mean)/(stdev*stdev)
        self.beta=self.tau/mean 
        self.min=0
        print(self.name, "(mean=", self.mean, ", stdev=", self.stdev, ", tau=", self.tau, ", beta=", self.beta, ")") 

    def getPDF(self, x)-> float: 
      if x>self.min:
            return gamma.pdf(self.beta*x, self.tau)*self.beta 
      else:
            return 0 

    def getCDF(self, x)->float: 
        if x>self.min:
            return gamma.cdf(self.beta*x, self.tau)
        else:
            return 0

    def getSDF(self, x)->float:
      if x>self.min:
            return 1-gamma.cdf(self.beta*x, self.tau )
      else:
            return 1 
        
    def lowerPercentile(self, p) -> float:
      return gamma.ppf(p, self.tau)/self.beta 



In [7]:





def get_retained_risk(x,a,b) ->float:
    if x<a:
        return x 
    elif x<b:
        return a 
    else:
        return x-(b-a)


def get_insured_risk(x,a,b) -> float:
    if x<a:
        return 0
    elif x<b: 
        return x-a 
    else: 
        return b-a 


def get_expected_risk(xx)->float:
    s: float =0
    for i in range(len(xx)):
        s+=xx[i]
    return s/len(xx)


def get_expected_retained_risk(xx,a,b)->float:
    s: float=0
    for i in range(len(xx)):
        s+=get_retained_risk(xx[i], a,b)
    return s/ len(xx)


def get_expected_insured_risk(xx,a,b)->float:
    s=0
    for i in range(len(xx)):
        s+=get_insured_risk(xx[i], a, b)
    return s/len(xx)




def getCCount(xx1,aa1,bb1,xx2,aa2,bb2) -> float:
    count=0
    for i in range(len(xx1)):
        rr: float=get_retained_risk(xx1[i], aa1,bb1)+get_retained_risk(xx2[i], aa2,bb2)
        if rr>aa1+aa2:
            count+=1
        return count 

def getCFraction(xx1, aa1, bb1, xx2, aa2, bb2)->float:
    count=0
    for i in range(len(xx1)):
        rr: float=get_retained_risk(xx1[i], aa1,bb1)+ get_retained_risk(xx2[i], aa2, bb2)
        if rr>aa1+bb2:
            count +=1
    return count/len(xx1)







In [8]:










from math import *
# from dist import *
# from insureutils import *

import matplotlib.pyplot as plt 
import numpy as np 


seed_num=135246780
num_points=1000000
num_iter=50
epsilon=0.000001


print_results=False 
print_counter=True 
plot_scatter=True 

alpha=0.01 
gamma=0.10 
theta=0.20 
delta=0.00 

corr=0.8 
c1=0.5*(sqrt(1+corr)+sqrt(1-corr))
c2=0.5*(sqrt(1+corr)-sqrt(1-corr))

rho=0.05 


mean1: float=50
stdev1: float=50 
mean2: float=50 
stdev2: float=50

dist1=Lognormal(mean1, stdev1)
dist2=Lognormal(mean2, stdev2)










probD=1-(1-rho)*(1-rho)
probE=(1-rho)*(1-rho)

seed(seed_num)

A=1/(1+theta)
a1=dist1.getUpperPercentile(A)
a2=dist2.getUpperPercentile(A)

print("-------")
print("a1= ", a1, ", a2= ", a2)






print("-------")
print("Generate risks sampling from the set D")

x1=np.zeros(num_points)
x2=np.zeros(num_points)

for i in range(num_points):
    u=bi_uniform_D(rho)
    g1=gaussian_invcdf(u[0])
    g2=gaussian_invcdf(u[1])
    h1=c1*g1+c2*g2
    h2=c1*g2+c2*g1
    v1=gaussian_cdf(h1)
    v2=gaussian_cdf(h2)
    x1[i]=dist1.getLowerPercentile(v1)
    x2[i]=dist2.getLowerPercentile(v2)

expectedRiskD1=probD*(get_expected_risk(x1))
expectedRiskD2=probD*(get_expected_risk(x2))
expectedRiskD=expectedRiskD1+expectedRiskD2 
print("--------")
print("expectedRiskD=", expectedRiskD)






print("--------")
print("Generate risks sampling from the set E")

y1=np.zeros(num_points)
y2=np.zeros(num_points)

for i in range(num_points):
    u=bi_uniform_E(rho)
    g1=gaussian_invcdf(u[0])
    g2=gaussian_invcdf(u[1])
    h1=c1*g1+c2*g2 
    h2=c1*g2+c2*g1 
    v1=gaussian_cdf(h1)
    v2=gaussian_cdf(h2)
    y1[i]=dist1.getLowerPercentile(v1)
    y2[i]=dist2.getLowerPercentile(v2)

insuredRiskE1=probE*(get_expected_insured_risk(y1, a1, np.inf))
insuredRiskE2=probE*(get_expected_insured_risk(y2, a2, np.inf))
insuredRiskE=insuredRiskE1+insuredRiskE2 


expectedRiskE=probE*(get_expected_risk(y1))+probE*(get_expected_risk(y2))

print("--------")
print("expectedRiskE =", expectedRiskE, ", expectedRiskDE =", expectedRiskD+expectedRiskE,)






print("--------")
print("Determine B1_max and b1_min")

b1=dist1.getUpperPercentile(A/2)
b2=dist2.getUpperPercentile(0)

B1_L=alpha 
B1_U=alpha/A 
B1=(B1_L+B1_U)/2 

while B1_U - B1_L > epsilon:
    probC=getCFraction(x1, a1, b1, x2, a2, b2)*probD 
    if probC>alpha:
        B1_U=B1 
    else:
        b1_L=B1 
    B1=(B1_L + B1_U) / 2
    b1=dist1.getUpperPercentile(B1)


B1_max=max(B1, alpha)
b1_min=dist1.getUpperPercentile(B1_max)

countC=getCCount(x1, a1, b1_min, x2,a2,b2)

probC_given_D=countC/num_points 
probC=probC_given_D*probD 

print("--------")
print("P(D)=", probD, ", P(E)=", probE)
print("P(C|D)=", probC_given_D, ", P(C)=", probC)

print("--------")
print("b2=", b2, ", b1_min=", b1_min, ", B1_max=", B1_max)


if plot_scatter:
    C1=np.zeros(countC)
    C2=np.zeros(countC)
    AB1=np.zeros(num_points-countC)
    AB2=np.zeros(num_points-countC)

    j: int=0
    k: int=0

    for i in range(num_points):
        rr: float=get_retained_risk(x1[i], a1, b1_min)+get_retained_risk(x2[i], a2,b2)
        if rr>a1+a2:
            C1[j]=x1[i]
            C2[j]=x2[i]
            j+=1
        else:
            AB1[k]=x1[i]
            AB2[k]=x2[i]
            k+=1

    plt.scatter(AB1,AB2, c='blue')
    plt.scatter(C1,C2, c='red')
    plt.show()






print("--------")
print("Determine B2_max and b2_min")

b1=dist1.getUpperPercentile(0)
b2=dist2.getUpperPercentile(A/2)

B2_L=alpha 
B2_U=alpha/A 
B2=(B2_L+B2_U)/2

while B2_U - B2_L>epsilon:
    probC=getCFraction(x1,a1,b1,x2,a2,b2)*probD 
    if probC>alpha:
        B2_U=B2 
    else:
        B2_L=B2 
    B2=(B2_L+B2_U)/2
    b2=dist2.getUpperPercentile(B2)

B2_max=max(B2, alpha)
b2_min=dist2.getUpperPercentile(B2_max)

countC=getCCount(x1, a1, b1, x2, a2, b2_min)

probC_givenD=countC/num_points 
probC=probC_given_D*probD 

print("--------")
print("P(D)=", probD, ",P(E)=", probE)
print("p(C|D)=", probC_given_D, ",P(C)", probC)

print("--------")
print("b1=", b1, "b2_min=", b2_min, ", B2_max=", B2_max )



if plot_scatter:
    C1=np.zeros(countC)
    C2=np.zeros(countC)
    AB1=np.zeros(num_points-countC)
    AB2=np.zeros(num_points-countC)

    j: int=0
    k: int=0

    for i in range(num_points):
        rr: float=get_retained_risk(x1[i], a1, b1)+get_retained_risk(x2[i], a2, b2_min)
        if rr>a1+a2 :
            C1[j]=x1[i]
            C2[j]=x2[i]
            j+=1 
        else:
            AB1[k]=x1[i]
            AB2[k]=x2[i]
            k+=1

    plt.scatter(AB1, AB2, c='blue')
    plt.scatter(C1,C2, c='red')
    plt.show()






print("--------")
print("Determine optimal b2 given b1")

BB1=np.zeros(num_iter)
BB2=np.zeros(num_iter)

PC=np.zeros(num_iter)
VAR=np.zeros(num_iter)

opt_B1=0
opt_B2=0
minVAR=np.inf 



for ii in range(num_iter):
    if print_counter: 
        print(".", end="")
    B1=B1_max*ii/(num_iter-1)
    b1=dist1.getUpperPercentile(B1)

    B2_L=(alpha-B1)/(1-B1)
    B2_U=(alpha-A*B1)/(A-B1)
    B2=(B2_L+B2_U)/2
    b2=dist2.getUpperPercentile(B2)

    while B2_U-B2_L>epsilon:
        probC=getCFraction(x1,a1,b1,x2,a2,b2)*probD 
        if probC>alpha: 
            B2_U=B2 
        else:
            B2_L=B2 
        B2=(B2_L+B2_U)/2
        b2=dist2.getUpperPercentile(B2)

    BB1[ii]=B1 
    BB2[ii]=B2 

    PC[ii]=getCFraction(x1, a1, b1, x2, a2, b2)*probD 

    insuredRiskD=probD*(get_expected_insured_risk(x1,a1,b1)+ get_expected_insured_risk(x2, a2, b2))

    VAR[ii]=a1+a2+(1+theta)*(insuredRiskD+insuredRiskE)

    if VAR[ii]<minVAR:
        opt_B1=B1 
        opt_B2=B2 
        minVAR=VAR[ii]
    
    if print_results:
        print("B1=", BB1[ii], ", B2=", BB2[ii], ",P(C)=", PC[ii], ", VAR=", VAR[ii])


opt_b1=dist1.getUpperPercentile(opt_B1)
opt_b2=dist1.getUpperPercentile(opt_B2)
insuredRiskD1=probD*(get_expected_insured_risk(x1, a1, opt_b1))
insuredRiskD2=probD*(get_expected_insured_risk(x2, a2, opt_b2))

print("opt_b1=", opt_b1, ", opt_b2=", opt_b2, ", opt_B1=", opt_B1, ", opt_B2=", opt_B2, ",minVAR=", minVAR,
", PI1=", (1+theta)*(insuredRiskD1+insuredRiskE1),
",PI2=", (1+theta)*(insuredRiskD2+insuredRiskE2))




plt.plot(BB1, BB2, label='Constraint')
plt.xlabel('B1')
plt.ylabel('B2')
plt.title('Constraint')
plt.legend()
plt.show()

plt.plot(BB1, PC, label='P(C)')
plt.xlabel('B1')
plt.ylabel('P(C)')
plt.title('p(C) versus B1')
plt.legend()
plt.show()


plt.plot(BB1, VAR, label='Value-at-risk')
plt.xlabel('B1')
plt.ylabel('V@R')
plt.title('V@R as a function of B1')
plt.legend()
plt.show() 





LOGNORMAL (mean= 50 , stdev= 50 , logmean= 3.5654494151481733 , logstdev= 0.8325546111576977 )
LOGNORMAL (mean= 50 , stdev= 50 , logmean= 3.5654494151481733 , logstdev= 0.8325546111576977 )
-------
a1=  28.290653445743114 , a2=  28.290653445743114
-------
Generate risks sampling from the set D
--------
expectedRiskD= 24.276839126715565
--------
Generate risks sampling from the set E
--------
expectedRiskE = 94.64390164622944 , expectedRiskDE = 118.92074077294501
--------
Determine B1_max and b1_min


KeyboardInterrupt: ignored

In [ ]:
 while True:pass